## Build RNN from scratch

### Étape 1 : Préparer les données
- Collecte de texte : Choisissez un texte à utiliser comme données d'entraînement (par exemple, un roman ou une collection de poèmes).
Traitement des données :
- Convertir les caractères en entiers (indexation des caractères).

In [11]:
text = "exemple de texte pour entraîner un RNN."  # Données
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}
char_to_idx

{' ': 0,
 '.': 1,
 'N': 2,
 'R': 3,
 'a': 4,
 'd': 5,
 'e': 6,
 'l': 7,
 'm': 8,
 'n': 9,
 'o': 10,
 'p': 11,
 'r': 12,
 't': 13,
 'u': 14,
 'x': 15,
 'î': 16}

- 2 - **Créer des séquences d'entrée-sortie (par exemple, une séquence de caractères comme entrée et le caractère suivant comme sortie).**


In [12]:
seq_length = 5  # Longueur de la séquence
inputs = []  # Contiendra les séquences d'entrée
targets = []  # Contiendra les sorties correspondantes

for i in range(len(text) - seq_length):
    inputs.append(text[i:i + seq_length])  # Séquence de 5 caractères
    targets.append(text[i + seq_length])  # Caractère suivant

# Exemple de résultat
print("Entrée:", inputs[:5])
print("Sortie:", targets[:5])


Entrée: ['exemp', 'xempl', 'emple', 'mple ', 'ple d']
Sortie: ['l', 'e', ' ', 'd', 'e']


- 3- **One hot encoding de chaque caractére**

In [19]:
def one_hot_dict(vocab_size:int, chars:list):
    """One-hot encoding of a each character in the list."""
    dict_one = {}
    for i, char in enumerate(chars):
        encoding = [0] * vocab_size
        encoding[i] = 1
        dict_one[char] = encoding
    return dict_one

one_hot_dict(len(chars), chars)

{' ': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 '.': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'N': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'R': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'a': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'd': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'e': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'l': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'm': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 'n': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 'o': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 'p': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 'r': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 't': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 'u': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 'x': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 'î': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1


### **3 : Initialiser les paramètres du RNN**

Un RNN utilise des matrices de poids pour transformer les données d'entrée et les états cachés en sorties. Voici les **éléments principaux** qu'on doit initialiser :

1. **Poids d'entrée vers l'état caché (\(W_{xh}\))** :
   - Transforme l'entrée (vecteur one-hot) en une contribution à l'état caché.

2. **Poids de récurrence (\(W_{hh}\))** :
   - Combine l'état caché précédent pour calculer le nouvel état caché.

3. **Poids de l'état caché vers la sortie (\(W_{hy}\))** :
   - Transforme l'état caché actuel en une probabilité pour chaque caractère (softmax).

4. **Biais (\(b_h\) et \(b_y\))** :
   - Valeurs ajoutées respectivement à l'état caché et à la sortie pour ajuster les calculs.

5. **État caché initial (\(h_0\))** :
   - Débute généralement avec un vecteur nul.



In [17]:
import numpy as np 

hidden_size = 50
vocab_size = len(chars)

# Initialisation des poids
W_xh = np.random.randn(hidden_size, vocab_size) * 0.01
W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
W_hy = np.random.randn(vocab_size, hidden_size) * 0.01

# Initialisation des biais
b_h = np.zeros((hidden_size, 1))
b_y = np.zeros((vocab_size, 1))

# État caché initial
hprev = np.zeros((hidden_size, 1))


Bien joué pour l'initialisation ! Maintenant, nous passons à **l'étape clé** : **la propagation vers l’avant (forward pass)**.

---

### **Étape 4 : Propagation vers l’avant**

#### **Objectif**
Calculer la sortie du RNN pour une séquence donnée :
1. Prendre chaque caractère de la séquence d'entrée (one-hot encodé).
2. Mettre à jour l'état caché (\(h_t\)) en fonction de l'entrée actuelle et de l'état précédent.
3. Calculer la probabilité de sortie (\(y_t\)) pour prédire le prochain caractère.

#### **Pourquoi ?**
La propagation vers l’avant permet au RNN de :
- Combiner les informations passées (via \(h_t\)).
- Produire une probabilité pour chaque caractère du vocabulaire en sortie.

---

### **Formules principales**
1. **Mise à jour de l'état caché** :
   \[$
   h_t = \tanh(W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h)
   $\]

2. **Calcul de la sortie** :
   \[$
   y_t = \text{softmax}(W_{hy} \cdot h_t + b_y)
   $\]

3. **Softmax** (convertir les scores en probabilités) :
   \[$
   y_t[i] = \frac{e^{z[i]}}{\sum_{j} e^{z[j]}}
   $\]

---

### **Étape à suivre pour coder cela**
1. **Initialiser l'état caché** (\(h_0\)) avec `hprev`.
2. **Pour chaque caractère** dans la séquence d'entrée :
   - Multiplier \(x_t\) par \(W_{xh}\).
   - Ajouter la contribution de l'état précédent (\(W_{hh} \cdot h_{t-1}\)).
   - Ajouter le biais (\(b_h\)) et appliquer `tanh` pour obtenir \(h_t\).
   - Calculer la sortie brute (\(W_{hy} \cdot h_t + b_y\)).
   - Appliquer la softmax pour obtenir les probabilités.

3. **Retourner les états cachés (\(h_t\)) et les probabilités (\(y_t\))**.



In [20]:
def forward_pass(inputs, hprev, W_xh, W_hh, W_hy, b_h, b_y, vocab_size, one_hot_dict):
    h = hprev  # État caché initial
    outputs = []  # Stocker les probabilités de sortie
    hs = {}  # Stocker les états cachés pour chaque étape
    hs[-1] = np.copy(h)  # État caché initial
    
    for t, char in enumerate(inputs):
        # Encoder le caractère en one-hot
        x_t = np.array(one_hot_dict[char]).reshape(-1, 1)
        
        # Calculer le nouvel état caché
        h = np.tanh(np.dot(W_xh, x_t) + np.dot(W_hh, h) + b_h)
        hs[t] = np.copy(h)
        
        # Calculer la sortie brute et appliquer softmax
        y_raw = np.dot(W_hy, h) + b_y
        y = np.exp(y_raw) / np.sum(np.exp(y_raw))  # Softmax
        outputs.append(y)
    
    return outputs, hs


Une fois la propagation vers l'avant implémentée et testée, l'étape suivante est d'implémenter la **propagation arrière** (**Backpropagation Through Time, ou BPTT**) pour permettre au RNN d'apprendre à partir des erreurs. C'est là que le RNN ajuste ses poids et biais pour mieux prédire à l'avenir.

---

### **Étape 5 : Propagation arrière (BPTT)**

#### **Pourquoi ?**
La propagation arrière permet de calculer les gradients des pertes par rapport aux paramètres du modèle (\(W_{xh}\), \(W_{hh}\), \(W_{hy}\), \(b_h\), \(b_y\)) afin d'effectuer une mise à jour avec un algorithme comme la descente de gradient.

#### **Déroulement de la BPTT**
1. **Calculer la perte globale** :
   - Pour chaque étape, compare les sorties prédites (\(y_t\)) aux cibles réelles (c'est-à-dire le caractère suivant attendu).
   - Utilise une fonction de perte, par exemple, **l'entropie croisée** :
     \[
     \text{loss} = -\sum_t \log(y_t[cible])
     \]

2. **Calculer les gradients** :
   - Remonte à travers le temps (de \(T\) à \(0\)) pour calculer les contributions de chaque étape à la perte globale.
   - Propager les gradients des erreurs dans :
     - Les sorties (\(y_t\)).
     - L'état caché (\(h_t\)).
     - Les poids (\(W_{xh}\), \(W_{hh}\), \(W_{hy}\)) et les biais.

3. **Mettre à jour les paramètres** :
   - Applique les gradients pour ajuster les poids et les biais via la **descente de gradient** :
     \[
     \theta \leftarrow \theta - \eta \cdot \frac{\partial \text{loss}}{\partial \theta}
     \]
   - (\(\eta\) est le taux d'apprentissage).

---

### **Étapes pour le code**
1. **Calculer la perte** :
   Implémente la formule d'entropie croisée pour chaque étape.

2. **Rétropropager les gradients** :
   Calcule les dérivées partielles pour chaque paramètre :
   - Les poids d'entrée (\(W_{xh}\)).
   - Les poids récurrents (\(W_{hh}\)).
   - Les poids de sortie (\(W_{hy}\)).
   - Les biais (\(b_h\), \(b_y\)).

3. **Effectuer une mise à jour des paramètres**.



In [21]:
def backward_pass(inputs, targets, outputs, hs, W_xh, W_hh, W_hy, b_h, b_y, vocab_size, one_hot_dict, learning_rate):
    # Initialiser les gradients
    dW_xh, dW_hh, dW_hy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
    db_h, db_y = np.zeros_like(b_h), np.zeros_like(b_y)
    dh_next = np.zeros_like(hs[0])  # Gradient de l'état caché suivant
    
    # Boucle inverse (du dernier caractère au premier)
    for t in reversed(range(len(inputs))):
        # Calcul de l'erreur de sortie
        dy = np.copy(outputs[t])
        target_idx = char_to_idx[targets[t]]
        dy[target_idx] -= 1  # Gradient de la perte par rapport à la sortie
        
        # Gradient pour W_hy et b_y
        dW_hy += np.dot(dy, hs[t].T)
        db_y += dy
        
        # Gradient de l'état caché
        dh = np.dot(W_hy.T, dy) + dh_next  # Gradient total pour h_t
        dh_raw = (1 - hs[t] ** 2) * dh  # Gradient après tanh
        
        # Gradient pour W_xh, W_hh, et b_h
        dW_xh += np.dot(dh_raw, one_hot_dict[inputs[t]].reshape(1, -1))
        dW_hh += np.dot(dh_raw, hs[t - 1].T)
        db_h += dh_raw
        
        # Propager le gradient vers l'état caché précédent
        dh_next = np.dot(W_hh.T, dh_raw)
    
    # Clipping des gradients (pour éviter exploding gradients)
    for dparam in [dW_xh, dW_hh, dW_hy, db_h, db_y]:
        np.clip(dparam, -5, 5, out=dparam)
    
    # Mise à jour des paramètres
    W_xh -= learning_rate * dW_xh
    W_hh -= learning_rate * dW_hh
    W_hy -= learning_rate * dW_hy
    b_h -= learning_rate * db_h
    b_y -= learning_rate * db_y
    
    return W_xh, W_hh, W_hy, b_h, b_y


### Entraînement du modèle
L'entraînement consiste à itérer plusieurs fois sur les données (les époques), appliquer la propagation avant pour obtenir les sorties du RNN, calculer la perte, et mettre à jour les poids avec la rétropropagation.

- **Objectif** :

Entraîner le RNN sur plusieurs époques (par exemple, 100 époques).
À chaque époque, passer par toutes les séquences de texte, calculer la perte et ajuster les poids.
- **Pourquoi ?**
L'entraînement permet au RNN d'améliorer ses prédictions en ajustant progressivement ses paramètres (poids et biais). Chaque époque aide le modèle à mieux comprendre la structure du texte pour générer des séquences plus cohérentes.

In [22]:
def train_rnn(inputs, targets, vocab_size, one_hot_dict, n_epochs, learning_rate):
    W_xh, W_hh, W_hy, b_h, b_y = initialize_parameters(vocab_size)  # Initialise les paramètres
    for epoch in range(n_epochs):
        loss = 0  # Pour suivre la perte de chaque époque
        hprev = np.zeros((hidden_size, 1))  # Réinitialise l'état caché pour chaque époque
        
        for i in range(len(inputs)):
            # Propagation avant
            outputs, hs = forward_pass(inputs[i], hprev, W_xh, W_hh, W_hy, b_h, b_y, vocab_size, one_hot_dict)
            
            # Calcul de la perte (entropie croisée)
            target_idx = char_to_idx[targets[i]]
            loss += -np.log(outputs[-1][target_idx])
            
            # Rétropropagation
            W_xh, W_hh, W_hy, b_h, b_y = backward_pass(inputs[i], targets[i], outputs, hs, W_xh, W_hh, W_hy, b_h, b_y, vocab_size, one_hot_dict, learning_rate)
        
        # Affichage de la perte à chaque époque
        print(f'Époque {epoch+1}/{n_epochs}, Perte : {loss}')
    
    return W_xh, W_hh, W_hy, b_h, b_y


## Génération de texte
Une fois que le modèle est entraîné, tu peux l'utiliser pour générer du texte à partir d'un caractère initial.

Objectif : Générer une séquence de texte de longueur 𝐿
L en partant d'un caractère initial et en utilisant les sorties du RNN pour prédire le caractère suivant.

Pourquoi ?
La génération de texte permet de tester la capacité du modèle à apprendre des dépendances et à prédire des caractères cohérents à partir du texte appris.

Comment ?
Initialiser l'état caché avec un vecteur nul ou un état caché appris.
Donner un caractère initial comme entrée.
À chaque étape :
Prédire le prochain caractère en fonction de l'état caché.
Utiliser ce caractère comme entrée pour la prédiction suivante.

In [23]:
def generate_text(start_char, length, W_xh, W_hh, W_hy, b_h, b_y, one_hot_dict, idx_to_char):
    hprev = np.zeros((hidden_size, 1))  # Initialisation de l'état caché
    generated_text = start_char  # Commencer avec le caractère initial
    
    for i in range(length):
        x_t = np.array(one_hot_dict[start_char]).reshape(-1, 1)  # Encoder le caractère initial
        h = np.tanh(np.dot(W_xh, x_t) + np.dot(W_hh, hprev) + b_h)  # Propagation avant
        
        # Calcul de la sortie et application de softmax
        y_raw = np.dot(W_hy, h) + b_y
        y = np.exp(y_raw) / np.sum(np.exp(y_raw))  # Softmax
        
        # Choisir le caractère suivant (maximum de la probabilité)
        next_char_idx = np.argmax(y)
        next_char = idx_to_char[next_char_idx]
        
        # Ajouter le caractère à la séquence générée
        generated_text += next_char
        start_char = next_char  # Utiliser le prochain caractère comme entrée pour la suivante génération
        hprev = h  # Mettre à jour l'état caché
    
    return generated_text
